In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
#x->(m,3) y->(m,)
y_train = torch.LongTensor([2,2,2,1,1,1,0,0])

x_test = torch.FloatTensor([[2,1,1],[3,1,2],[3,3,4]])
y_test = torch.LongTensor([2,2,2])

class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,3)
    def forward(self, x) :     #x_train->(m,3)  ->  x = 3
        return self.linear(x)


model = SoftmaxClassifierModel()
optimizer = optim.SGD( model.parameters(), lr=0.1 )

def train(model, optimizer, x_train, y_train) :
    nb_epochs = 20
    for epoch in range(nb_epochs):
        prediction = model(x_train)
        cost = F.cross_entropy(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch  {:4d}/{} Cost: {:.6f}'.format( epoch, nb_epochs, cost.item()))

def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)

    print('Accuracy: {}% Cost: {:.6f}'.format( correct_count / len(y_test) * 100, cost.item()))


train (model, optimizer, x_train, y_train)
test (model, optimizer, x_test, y_test)

#이미 오버피팅 된 상태

Epoch     0/20 Cost: 2.600954
Epoch     1/20 Cost: 1.829004
Epoch     2/20 Cost: 1.220283
Epoch     3/20 Cost: 1.118137
Epoch     4/20 Cost: 1.103364
Epoch     5/20 Cost: 1.091616
Epoch     6/20 Cost: 1.081515
Epoch     7/20 Cost: 1.072420
Epoch     8/20 Cost: 1.063893
Epoch     9/20 Cost: 1.055778
Epoch    10/20 Cost: 1.047952
Epoch    11/20 Cost: 1.040358
Epoch    12/20 Cost: 1.032957
Epoch    13/20 Cost: 1.025728
Epoch    14/20 Cost: 1.018653
Epoch    15/20 Cost: 1.011726
Epoch    16/20 Cost: 1.004938
Epoch    17/20 Cost: 0.998284
Epoch    18/20 Cost: 0.991762
Epoch    19/20 Cost: 0.985366
Accuracy: 0.0% Cost: 1.092040


## Overshooting
learning rate가 너무 크면 diverge 하면서 cost가 점점 늘어난다. (7-1 37분쯤 그림 참고)

In [15]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD( model.parameters(), lr = 1e5)
train(model, optimizer, x_train, y_train)


Epoch     0/20 Cost: 1.519528
Epoch     1/20 Cost: 1050876.875000
Epoch     2/20 Cost: 1327161.250000
Epoch     3/20 Cost: 684830.937500
Epoch     4/20 Cost: 1413376.750000
Epoch     5/20 Cost: 1530286.250000
Epoch     6/20 Cost: 977439.312500
Epoch     7/20 Cost: 774344.250000
Epoch     8/20 Cost: 606520.312500
Epoch     9/20 Cost: 1547751.875000
Epoch    10/20 Cost: 1614661.250000
Epoch    11/20 Cost: 1111814.375000
Epoch    12/20 Cost: 823384.000000
Epoch    13/20 Cost: 861393.437500
Epoch    14/20 Cost: 1452439.375000
Epoch    15/20 Cost: 1170911.250000
Epoch    16/20 Cost: 1019531.875000
Epoch    17/20 Cost: 635042.062500
Epoch    18/20 Cost: 592257.312500
Epoch    19/20 Cost: 1594626.750000


너무 작으면 학습이 너무 느림(cost의 변화가 거의 없음)

In [16]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD( model.parameters(), lr = 1e-10)
train(model, optimizer, x_train, y_train)

Epoch     0/20 Cost: 2.350477
Epoch     1/20 Cost: 2.350477
Epoch     2/20 Cost: 2.350477
Epoch     3/20 Cost: 2.350477
Epoch     4/20 Cost: 2.350477
Epoch     5/20 Cost: 2.350477
Epoch     6/20 Cost: 2.350477
Epoch     7/20 Cost: 2.350477
Epoch     8/20 Cost: 2.350477
Epoch     9/20 Cost: 2.350477
Epoch    10/20 Cost: 2.350477
Epoch    11/20 Cost: 2.350477
Epoch    12/20 Cost: 2.350477
Epoch    13/20 Cost: 2.350477
Epoch    14/20 Cost: 2.350477
Epoch    15/20 Cost: 2.350477
Epoch    16/20 Cost: 2.350477
Epoch    17/20 Cost: 2.350477
Epoch    18/20 Cost: 2.350477
Epoch    19/20 Cost: 2.350477


## Data Preprocessing

In [18]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

mu = x_train.mean(dim=0)
sigma = x_train.std(dim=0)
norm_x_train = (x_train - mu) / sigma

print(norm_x_train)
# 어떤 정규분포를 만드는 trianing set 만들기

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


In [21]:
class MultivariateLinearRegressionModel(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.linear = nn.Linear(3,1)

    def forward(self, x) :
        return self.linear(x)

model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr = 1e-1)

def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs) :
        prediction = model (x_train)
        cost = F.mse_loss(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{}  Cost : {:.6f}'.format(epoch, nb_epochs, cost.item()))

train(model, optimizer, norm_x_train, y_train)

# y_train이 이번엔 1차원이었지만 차원이 높아지고 첫번째와 두번째 값의 차이가 클 때
# 전처리 하지 않으면 큰 값에만 집중해서 cost를 줄여나감(작은 값은 영향이 작기 때문에)

Epoch    0/20  Cost : 29767.603516
Epoch    1/20  Cost : 18900.458984
Epoch    2/20  Cost : 12052.658203
Epoch    3/20  Cost : 7701.250977
Epoch    4/20  Cost : 4925.418457
Epoch    5/20  Cost : 3151.517090
Epoch    6/20  Cost : 2016.978516
Epoch    7/20  Cost : 1291.087158
Epoch    8/20  Cost : 826.571167
Epoch    9/20  Cost : 529.290161
Epoch   10/20  Cost : 339.025940
Epoch   11/20  Cost : 217.249313
Epoch   12/20  Cost : 139.303833
Epoch   13/20  Cost : 89.410202
Epoch   14/20  Cost : 57.470360
Epoch   15/20  Cost : 37.020805
Epoch   16/20  Cost : 23.925549
Epoch   17/20  Cost : 15.537231
Epoch   18/20  Cost : 10.161668
Epoch   19/20  Cost : 6.714508
